# Stream Cipher

In cryptography, a stream cipher is a symmetric key cipher that encrypts the plaintext one bit (or byte) at a time. 

A common key is exchanged between the two entities that want to exchange a message and starting from this, a keystream is generated on both sides which is then used to encrypt and decrypt the message by simply xoring this sequence with the message.

The keystream that is generated for both ecryption and decryption is a pseudorandom sequence which means that although it appears to be purely random, it is generated serially starting from a seed.

In [1]:
# import modules here
from itertools import islice
import utils
import functools
from operator import xor

## LFSR

An LFSR (Linear Feedback Shift Resiter) is a particular shift register where the input bit is generated by a linear combination of the status bits (implemented with XOR and AND logics). 

How an LFSR works can be described by 3 equations:
<p align = "center">
<img src="LFSRbasics.PNG" alt= “” width="35%" height="35%" >
</p>
Where the first equation represents the shift register, the second the linear feedback and the latter represents the output bit. 

Due to its structure it is capable of generating an infinite, periodic sequence. The length of this period depends only on the polynomial used to describe the linear feedback, in particular if this polynomial is irreducible and divides $(1-x)^T$ then we are able to generate a sequence with a maximum length of $2^m-1$ (where $m$ is the grade of the polynomial and $T$ is the maximum length of the sequence). 

In [5]:

class Lfsr_class:

    def __init__(self, poly, state=None):
        self._poly = poly
        self._state = state
        self.length = max(poly)
        self.poly = [1 if i in poly else 0 for i in range(self.length + 1)]

        if state is None:
            self.state = [1 for _ in range(self.length + 1)]
        else:
            self.state = [int(i) for i in list(bin(state)[2:])]
            self.state = [0]*(self.length - len(self.state)) + self.state if (self.length > len(self.state)) else self.state

        self.output = self.state[self.length - 1]
        self.feedback = functools.reduce(xor, [a & b for a, b in zip(self.poly[1:], self.state)])

    def __len__(self):
        return self.length

    def __iter__(self):
        return self

    # this runs every single step of a N length LSFR
    def run_steps(self, n=0):
        return [next(self) for _ in range(n)]

    # this executes a full LFSR cycle
    # if length is 3, then it performs 2^3-1 steps
    def cycle(self, state=None):
        if state is not None:
            self.state = state
        return self.run_steps(2 ** (len(self)) - 1)

    def __next__(self):
        self.state.insert(0, self.feedback)
        self.state = self.state[:self.length]
        self.output = self.state[self.length - 1]
        self.feedback = functools.reduce(xor, [a & b for a, b in zip(self.poly[1:], self.state)])

        return self.output

    def __str__(self):
        string = f'{self.state} {self.output} {self.feedback}'
        return string

    def __repr__(self):
        string = f'{self.state} {self.output} {self.feedback}'
        return string


### testing

The LFSR class was implemented as an iterator, this is because it is a shift register and we wish to iterate it several times. The following attributes are the ones describing each instance of the class: 
- ***poly*** list of 1 or 0 representing the polynomial that describes the LFSR
- ***state*** current state of the internal ffs
- ***length*** number of internarl ffs and degree of the polynomial 
- ***output*** value of the output bit
- ***feedback*** value of the feedback bit computed using the current state and the polynomial

The most important methods that we provided to the class are: *\_\_init\_\_* (the constructor of the object), *\_\_iter\_\_* is used to define the class as an iterable, the *\_\_next\_\_* method is called whenever the object is iterated and it updates the LFSR state and returns the output bit, *\_\_cycle\_\_* used to run a maximum length sequence and *runsteps* that run every single step. 

Prove class functionality (test every method) in these three cases:
- `poly = [3, 1, 0]` and `state = 6` (`0b110`)
- `poly = [5, 2, 1, 0]` and `state = 5` (`0b00101`)
- `poly = [9, 5, 0]` and `state = 511` (`0b111111111`)`

In [6]:
def lfsr_generator(poly_list, state_list, berk = False):
    
    lfsr = Lfsr_class(poly_list, state_list)
    
    if not berk :
        print('\nstate     b fb')
        for _ in islice(lfsr, 10):
            print(lfsr)

    if berk : 
        full_cycle = lfsr.cycle()
        print(f'\nfull LFSR cycle -> {full_cycle}')

        P, comp = berlekamp_massey(full_cycle)

        print_poly(P)

In [7]:
# Code here
poly_list = [3, 1, 0] 
state_list = 7
lfsr_generator(poly_list, state_list)


state     b fb
[0, 1, 1] 1 1
[1, 0, 1] 1 0
[0, 1, 0] 0 0
[0, 0, 1] 1 1
[1, 0, 0] 0 1
[1, 1, 0] 0 1
[1, 1, 1] 1 0
[0, 1, 1] 1 1
[1, 0, 1] 1 0
[0, 1, 0] 0 0


In [8]:
poly_list = [5, 2, 1, 0] 
state_list = 5
lfsr_generator(poly_list, state_list)


state     b fb
[1, 0, 0, 1, 0] 0 1
[1, 1, 0, 0, 1] 1 1
[1, 1, 1, 0, 0] 0 0
[0, 1, 1, 1, 0] 0 1
[1, 0, 1, 1, 1] 1 0
[0, 1, 0, 1, 1] 1 0
[0, 0, 1, 0, 1] 1 1
[1, 0, 0, 1, 0] 0 1
[1, 1, 0, 0, 1] 1 1
[1, 1, 1, 0, 0] 0 0


In [9]:
poly_list = [9, 5, 0] 
state_list = 511
lfsr_generator(poly_list, state_list)


state     b fb
[0, 1, 1, 1, 1, 1, 1, 1, 1] 1 0
[0, 0, 1, 1, 1, 1, 1, 1, 1] 1 0
[0, 0, 0, 1, 1, 1, 1, 1, 1] 1 0
[0, 0, 0, 0, 1, 1, 1, 1, 1] 1 0
[0, 0, 0, 0, 0, 1, 1, 1, 1] 1 1
[1, 0, 0, 0, 0, 0, 1, 1, 1] 1 1
[1, 1, 0, 0, 0, 0, 0, 1, 1] 1 1
[1, 1, 1, 0, 0, 0, 0, 0, 1] 1 1
[1, 1, 1, 1, 0, 0, 0, 0, 0] 0 0
[0, 1, 1, 1, 1, 0, 0, 0, 0] 0 1


In order to semplify the rappresentation of the result we only print the firt 10 iterations with the method *islice()*. 

An analysis of the three sequences have showed the following characteristics:
- The first polynomial produces a maximum length period equal to 7, hence it is a primitive polynomial.
- The second polynomial generates a period that is equal to 7, however since its degree is 5 then it is not a primitive polynomial. 
- The third polynomial generates a period equal to 2^N-1 where N is its degree that is equal to 9, hence it is a primitive polynomial.

## Berlekamp Massey

The Berlekamp Massey algorithm is an algorithm that given an input binary sequence of arbitrary length finds the shortest LFSR capable of generating it.  The BM algorithm is the main reason why LFSRs are not used directly as pseudo-random number generators since they are vulnerable to KPA.   

In [10]:
def print_poly(P):
    P.reverse()
    nonzero = [i for i, e in enumerate(P) if e != 0]
    poly_list = [f'x^{d}' for d in nonzero if d!= 0]
    poly_list.reverse()
    poly = "+".join(poly_list) + "+1"
    print(poly)

    

In [11]:
def berlekamp_massey(bits):
    m, r, P, Q = 0, 1, 1, 1

    for t in range(len(bits)):
        d_prev = utils.bitlist_to_int(bits[t - m:t + 1]) & P
        d = utils.disparity(d_prev)

        if d == 1:
            if 2 * m <= t:
                R, P = P, P ^ (Q << r)
                Q = R
                m, r = t + 1 - m, 0
            else:
                P = P ^ (Q << r)

        r = r + 1

    linear_complexity = len(utils.int_to_bitlist(P)) - 1

    return utils.int_to_bitlist(P), linear_complexity

### testing

The algorithm has been defined within a function that takes as its only input a list of 1s and 0s representing the sequence to be analysed and returns the minimum polynomial that generates it and its degree. To simplify the program structure, the two polynomials P and Q are treated as integers instead of lists since operations such as shift and AND are much easier to implement with integers. First the disparity of the last m bits is calculated with the polynomial that currently describes the sequence, and depending on the value obtained, the variables P, Q, r, m are updated or not. 

A maximal length period is generated by lfsr_generator to check for correctness, after which it is sufficient to check that the generated polynomial is equal to the one passed. 

Prove its functionality in these three cases, using the already defined LFSR class:
- `poly = [3, 1, 0]` and `state = 6` (`0b110`)
- `poly = [5, 2, 1, 0]` and `state = 5` (`0b00101`)
- `poly = [9, 5, 0]` and `state = 511` (`0b111111111`)`

In [12]:
# Code here
poly_list = [3, 1, 0] 
state_list = 7
lfsr_generator(poly_list, state_list, True)


full LFSR cycle -> [1, 1, 0, 1, 0, 0, 1]
x^3+x^1+1


In [13]:
poly_list = [5,2,1,0] 
state_list = 5
lfsr_generator(poly_list, state_list, True)


full LFSR cycle -> [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0]
x^3+x^1+1


In [14]:
poly_list = [9,5,0] 
state_list = 7
lfsr_generator(poly_list, state_list, True)


full LFSR cycle -> [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1

The first and last sequences are generated by prime polynomials, whereas the second one is not hence we found a polynomial whose degree is smaller compared to the one that we expected. 

Given the following key stream `keystream`, generated by an LFSR, apply the Berlekamp-Massey Algorithm to discover the structure (in particular, the polynomial) of the LFSR.

In [15]:
keystream = b'\xfa\xb3v\x93\x8b\xca0\x83'

keylist = utils.bytes_to_bits(keystream)
# Convert keystream in a list of bits here:
P, m = berlekamp_massey(keylist)
# Run BM here:

print_poly(P)

x^6+x^1+1


## LFSR-based Generator

As mentioned earlier, BM's algorithm makes LFSR-based cipher streams vulnerable to KPA-type attacks. However, LFSRs can be used as building blocks for more complex generators as they are very fast and have good statistical properties.

An example is the Alternating step generator which is composed of three LFSRs and belongs to the family of irregular clocking generators. Depending on the output value of the LFSRc one of the two upstream LFSRs (LFSR0 and LFSR1) is activated, whereas the other remains in its previous state. The final output is the xor between the output bit of the two LFSRs at time t. 

- `poly0 = [5, 2, 0]`
- `poly1 = [3, 1, 0]`
- `polyC = [2, 1, 0]`

In [16]:
def _check_seed(seed, lfsr_type):
    if 1 in seed:
        return seed
    else:
        raise ValueError(f'The LFSR {lfsr_type} has its initial state set to 0')


def _bit_to_int(bitlist):
    out = 0
    for bit in bitlist:
        out = (out << 1) | bit
    return out


class AlternatingStepGenerator:
    def __init__(self, seed):

        seed_bin = [int(i) for i in bin(seed)[2:]]

        try:
            self.seed_c = _check_seed(seed_bin[8:], "C")
            self.seed_0 = _check_seed(seed_bin[:5], "0")
            self.seed_1 = _check_seed(seed_bin[5:8], "1")
        except ValueError as err:
            print(err)
            quit()

        self.lfsr_c = Lfsr_class([2, 1, 0], _bit_to_int(self.seed_c))
        self.lfsr_0 = Lfsr_class([5, 2, 0], _bit_to_int(self.seed_0))
        self.lfsr_1 = Lfsr_class([3, 1, 0], _bit_to_int(self.seed_1))

    def __iter__(self):
        return self

    def __next__(self):
        if next(self.lfsr_c):
            bit = next(self.lfsr_1) ^ self.lfsr_0.output
        else:
            bit = self.lfsr_1.output ^ next(self.lfsr_0)

        return bit

Compute the Linear complexity of the generated sequence.

In [17]:
# Code here
SEED = 0x3FF
alt_gen = AlternatingStepGenerator(SEED)

gen_seq = [next(alt_gen) for _ in range(1000)]

P,m = berlekamp_massey(gen_seq)

print_poly(P)
print(m)


x^24+x^18+x^6+x^3+1
24


<p align = "center">
<img src="alternating_step.png" alt= “” width="33%" height="33%" >
</p>

As mentioned in the work of M. M. Hassanzadeh and T. Helleseth, "Algebraic attack on the Alternating Step(r,s) Generator", the linear complexity of the output sequence of an alternating step generator is: $(m+n)2^(l-1)<L<(m+n)2^l$. Our result is 24 which is indeed comprised into the bounded set we can compute with the previous mentioned formula : $8<L<32$.



### testing

An AlternatingStepGenerator() class was created which only takes as input the seed, i.e. the configuration of the internal registers of the various LFSRs, the structure being assumed to be fixed. The seed is divided into three sub-seeds wich are used to initialize the LSFRs. The LSFRc is updated every cycle and its output is used to activate one of the two upstream LFSRs.  

Given the key `key`, decrypt the ciphertext `ciphertext`.

In [18]:
def Alternating_step():
    SEED = 0x3FF
    CIPHERTEXT_ALT_STEP = (
        b'Y\xea\xfc\xc2\x8c\x17p{\x1f8\xbf,N\\\xf8\x97\xeb\x99#\'#\xf3\x1cY\xfd'
        b'\x82\xe9\xbe\xc2\xeb\x16H\xd0Q\xd5\xa8Y\x8e\x8b\\\xeb\x8d\xe1\xea\xf5'
        b'\x83\xb0\xe7\xee\xf2\\\xear\x848l\xe2\xb2\x06ov%\xdb\x08\x13\xf2qy\xf6'
        b'}\xfdO\xebG{\xaa\xaf\xbaP\xc1\x98c\x19\xdc5\xf3\x00P_\x0e\x8b\xe9\xa5$'
        b'\xaf,7\x99G\xe5\x89f\xd5o\xd9s\xd4\x10;\xf4\x10\x1a\x84]\xf6>\xd9J\x86'
        b'\xb2/%\x86\x92\xdc3f\x1d\x15\xa0\xa7K1*\xa0\xaa\x88+x\xb9\xa9#Ie)\xf97'
        b'\x05\xf1\x1b\x02~\xeac\xd0\xeb\x0bv+\xd2Y\xb1\xbd\x1d6!\xde\xd9\x9de'
        b'\x05\xb3\xf30\x14\xfd\xa7s\xa9\xce\xad]\x01W\x0b\x9a\xb6\x03\x8e^J\xbf'
        b'\x01\xe4\xb6\xa0\x83\xbej\xb6\xda\xca\xa3J\xd78~1\xaf/t\x1eL\x93\x19J'
        b'\xb18Bm\x7f\xddZ\xa8L\xbd\xb1\x84,\x04\xc7\x0b,\x11\xce}\xed\x06\x06XZ'
        b'\xaf\xcf\xaa-\xaf\xad"\x87\xd3\xf8\xc2\xf7\xd3\x1bM}\xb9\x00)\xb1\x9a'
        b'\x06\xcdU\xedV\xd7\x03\x90\xed;>\xc5\xd7\xff\xa0qZ\x94\xf3\xb6\x1b9v'
        b'\xfa\xfa\xf1x}\xd9\xf3\x7fm\xe4 \xe0"G\xe0O*C%p\xd7yYS&\xd9{\xec\xe9'
        b'=46\xfbH\xcc#\x0f\xe8\xf78H\xcc *\xb8\xd8\xe35\xda\x03>\xc5\xf0\x1a'
        b'OCZ\xfc\x11\xbd\xf7\xb0\xc9\xb2!\xfe\xd8\xc7\x8e\x1c\xc3:\x7fb\xdd9wZ'
        b'\xad\xca\\\x83\xf9>Fx\x1dQ\x1d\x9a\x92\xdb\xc1\x8b+\x19\xdfDK\x93\xd7M'
        b'\xe7Cg\xdbP\xa6\x99\xe5`\xae\xed6E\xcf\xe3\xc2\xb5\xee\x80\x14D+@5\xb2'
        b'\xde\x02\xdb\x01\x9b\xd9\x90<\x00\xe6\n=\x98\xf6\xe9\xb7\x14\x93\x95'
        b'\xc8\xf7YX!\xe2\x830<q\x9b\xed\x034\xa0\x0c|(\x05%h3\x87dN\x160zN\''
        b'\x8ev\xe4\xe0\xb0q\x02\xb1\x10\xa0\x90\x06\xf42SSV4nl\xf4\xd8\xe1\xc3S'
        b'?\x89\xe5\x80\x11X\x1f\xfe-"\xed\xb4D\xb6a\xa3\xdd\xc8\xca\t\xfcrg\x0e'
        b'\xfa|X\x16\x82\xc2\xdb\x86\xfd=\x07cK\x15?\x98\xd3\xf8\xda\xcb\x0c\x0e'
        b'\x84\\\x9c\x84\x87\xd1\xa5P\xab\xcd;')

    alt_gen = AlternatingStepGenerator(SEED)

    # ciphertext expressed as a list of integers, where each integer is a byte
    cipher_list_int = utils.bytes_to_bits(CIPHERTEXT_ALT_STEP)
    plaintext = utils.bits_to_bytes([a ^ b for a, b in zip(cipher_list_int, alt_gen)])
    print(plaintext)

In [19]:
# Code here
Alternating_step()

b'Bologna (Latin: Bononia) is a city in and the capital of theEmilia-Romagna region in Northern Italy, of which it is alsoits largest. It is the seventh most populous city in Italywith about 400,000 inhabitants and 150 different nationalities.Its metropolitan area is home to more than 1,000,000 people.It is known as the Fat City for its rich cuisine, and the Red City for its Spanish-style red tiled rooftops and, more recently, its leftist politics. It is also called the Learned City because it is home to the oldest university in the world.'


## RC4

Unlike the other PRNGs, RC4 works with bytes (not bits) and bases its operations on the existence of a permutation. The algorithm consists of two parts: the first one called KSA where the permutation is initialised from a key(whose length is comprised between 5 and 255), the second one where there is the generation of the keystream and the modification of the permutation is called PRGA. 

The main vulnerability of this algorithm is related to the correlation between the key and the first bytes produced so to increase security usually the first n outputs are discarded. 

In [20]:
class RC4:

    def __init__(self, seed, drop=0):
        # initialization of the identity permutation
        self.P = list(range(256))
        t = 0

        for k in range(256):
            t = (t + self.P[k] + seed[k % len(seed)]) % 256
            self.P[k], self.P[t] = self.P[t], self.P[k]

        # variables for the PGRA
        self.i = 0
        self.j = 0

        # It'll drop the first n values generated
        for _ in range(drop):
            self.__next__()

    def __iter__(self):
        return self

    def __next__(self):
        self.i = (self.i + 1) % 256
        self.j = (self.j + self.P[self.i]) % 256
        self.P[self.i], self.P[self.j] = self.P[self.j], self.P[self.i]
        byte = self.P[(self.P[self.i] + self.P[self.j]) % 256]
        return byte

### testing

Whenever an object of the RC4 class is created the constructor method is called and its role is to compute the KSA algorithm that consists into defining a vector P with an identity permutation and then shuffling it according to the given key and also it drops the first N generated bytes to decrease the correlation between the key and the first bytes that are generated.

The class has been implemented as an iterator, hence whenever the *\_\_next__* method is called it computes one step of the PRGA algorithm.

Given the key `key`, decrypt the ciphertext `ciphertext`.

In [21]:
# Code here
def RC4_dec(key, ciphertext, drop):
    RC4_key = RC4(key, drop)
    plaintext = [chr(a ^ b) for a, b in zip(RC4_key, ciphertext)]
    print("".join(plaintext))

In [22]:
key = b'0123456789ABCDEF'
ciphertext = (
    b"\x0f\x9e\xec\xc3\x0f\xeck\xa5\xc9\xd4\xd4\xb81\xf0\xd5[\x93"
    b"\xbc\xdaY^\x0e\xc1'\xdb\x1eP\xc0uD\xeb\x91E]\x15\xcf\x85\x07%"
    b"\x97\xffl\xf1\xbb\xe1*\xa0\xee\xd3\x94\x17\xb0|e\x93h\xd7\xc5"
    b"\xc3tT\x84\xee%\xcb\x15\r\xc4\xe8\n\xf2\xd0.\xec\xc6\xe1-\xb4"
    b"\x8b\x9a\x14\x823k\xab?\x8b\x9c\xaas\xa1#\xb8\xb2\xceh\xb5\x8b"
    b"'\x90B}C\x80~\x8cr\xde\xc9\xe2\x17\xe45\xb9\x10\x94\xd4\x0eRJ"
    b"\x0fr&\xe7\xe3P\xbfz\xecIA\x94\xe60\xa8{_\x03\xc7\x91\xcf\xc6"
    b"\x04\xfc\x8d\x86-E\x13\xba\x13i\x17;\xd7\x8e \xa5\xe6\xa5uR\n"
    b"89z\xe2YZ!e\x0f,s\x9a\xacN\xab\xc7\xcaOO\x81\xe06\x03\xac7\x9b"
    b",%\xf7\x9d(\xde\x0b\xc3\xbf\xbe7\xc7<\xf4r\x0eLz\xd8\xe5b\xa0n"
    b"\x11f\xfe\xca+RNnL\xef+\x1b\x1c\x0b\x8a\xb6M\xbdU\xd6\x1c\x9cn"
    b"\t\x8aX>B9\xa4\xf7\xd7jS\xa3\xfd\xeb=\xeeY\xbf\x8dG\xab\xc6"
    b"\x08 \xfba\x90\xdbla\xbf\xf0\x9c\x1a\xa0\xcb\x9a\x7f\x88X\x1dV"
    b"\xd2'\xba\x1b\x16l\xc3vx\xddqU\xf72@\x86>A\xe6b\x050Y\xed\x8bJ"
    b"!\xe0\x80I\xa4X\x9e\xd6^\x126\x99\x93\xa3(\xc5\xf0\xbd\xdei\xcb"
    b"\xdc5\xf58\xda\xa5\x96\xb9!5>\xdbJ\xd4\xef\xe1\x0e\xfeo\x97`"
    b"\xd7[\x18\t\xf4/DV\xdd\xe8~#\xbd\xfd{B\xacC=\xa9\xd5\xbb\xdbH"
)
drop = 3072

RC4_dec(key, ciphertext, drop)

Hey, Macklemore, can we go thrift shopping?What? What? What? What?
What? What? What? What?
What? What? What? What?
What? What? What? What?
What? What? What? What?
What? What? What? What?
What? What? What? What?
What? What? What? What?
Oh!
Oh!
Ow!
I m gonna pop some keys
Only got 3072 drop in my RC4
I m, I m, I m hunting, looking for a come up
This is ******* awesome



## Bonus Task

Build an object implementing a Stream Cipher that, given a Pseudo Random Number Generator (PRNG) and a key, can encrypt and decrypt a message. Note that, PRNG can output either a bit (LFSR-based generator) or a byte (RC4).

We have created a general purpose class Stream cipher which is used to either encrypt or decrypt a plaintext or ciphertext. The keystream can be generated with a generic bit or byte generator. For this purpose we have implemented the constructor that takes the kwarg argument which enables us to pass an unknown number of variables.

In [23]:
class StreamCipher:

    def __init__(self, key, prng, **kwargs):
        self.prng = prng(key, **kwargs)

    def encrypt(self, plaintext):
        return self._crypting(plaintext)

    def decrypt(self, ciphertext):
        return self._crypting(ciphertext)

    def _crypting(self, text):
        return [a ^ b for a, b in zip(text, self.prng)]

### testing

The previous stream cipher class has been implemented with the previous defined RC4 byte generator where we are passing a third variable ndrop that is tackled by the kwarg argument. 

In [24]:
def bonus_task(MESSAGE, KEY, ndrop):

    alice = StreamCipher(KEY, prng=RC4, drop=ndrop)
    bob = StreamCipher(KEY, prng=RC4, drop=ndrop)

    ciphertext = alice.encrypt(MESSAGE.encode('utf-8'))
    plaintext_bob = bob.decrypt(ciphertext)
    print(f'Ciphertext: {ciphertext}')
    print(f'Decrypted message: {"".join([chr(i) for i in plaintext_bob])}')

In [25]:
MESSAGE = "\nNever gonna give you up \nNever gonna let you down\nNever make these two students fail the exam"
KEY = b"0123456789ABCDEF"
ndrop = 3072
bonus_task(MESSAGE, KEY, ndrop)

Ciphertext: [77, 181, 240, 153, 74, 211, 42, 161, 205, 214, 223, 180, 126, 229, 217, 1, 214, 255, 194, 88, 11, 89, 209, 119, 156, 123, 62, 209, 107, 83, 240, 215, 86, 18, 8, 201, 139, 87, 57, 155, 229, 43, 183, 131, 252, 107, 176, 190, 132, 173, 117, 159, 109, 44, 214, 77, 159, 201, 214, 32, 17, 243, 242, 44, 218, 89, 98, 179, 244, 28, 233, 207, 125, 207, 219, 228, 60, 229, 223, 190, 92, 133, 38, 61, 231, 72, 151, 149, 187, 108, 206, 12, 177, 190]
Decrypted message: 
Never gonna give you up 
Never gonna let you down
Never make these two students fail the exam
